In [346]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ISLP.models import ModelSpec as MS
import statsmodels.api as sm
import keras
from keras import layers
from keras import ops
from sklearn.model_selection import train_test_split

In [347]:
import os
PathToRepo = os.path.normpath(os.getcwd() + os.sep + os.pardir)
Df = pd.read_csv(PathToRepo + '\\Data\\Faellesdata_cleaned.csv')

x = Df.drop('IsMigratorInt', axis=1) 
for i in range(0, x.shape[1]): #Standardizing the data
    x.iloc[:,i] = (x.iloc[:, i] - x.iloc[:, i].mean())/x.iloc[:, i].std()

x_train, x_test, y_train, y_test = train_test_split(x, Df['IsMigratorInt'], test_size=0.2, random_state=42)

In [348]:
x_train = MS(x_train, intercept = False).fit_transform(x_train)

In [349]:
x_train.shape

(7999, 15)

In [350]:
modelnn = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [351]:
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=10)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 969us/step - accuracy: 0.7989 - loss: 0.4474
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 962us/step - accuracy: 0.8502 - loss: 0.3237
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 982us/step - accuracy: 0.8536 - loss: 0.3009
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step - accuracy: 0.8589 - loss: 0.2858
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step - accuracy: 0.8608 - loss: 0.2874
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8670 - loss: 0.2753
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 972us/step - accuracy: 0.8685 - loss: 0.2729
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 940us/step - accuracy: 0.8670 - loss: 0.2739
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 939us/step - accuracy: 0.8799 - loss: 0.2560
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 973us/step - accuracy: 0.8740 - loss: 0.2588


In [352]:
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8768 - loss: 0.2614  


[0.2641180455684662, 0.8694999814033508]

In [353]:
from sklearn.metrics import accuracy_score

In [354]:
print(accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
0.8695


In [355]:
modelnn.optimizer.get_config()['name']

'adam'

In [356]:
dir(modelnn.get_layer(index=0))

['__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_allow_non_tensor_positional_args',
 '_api_export_path',
 '_api_export_symbol_id',
 '_assert_input_compatibility',
 '_auto_config',
 '_build_by_run_for_kwargs',
 '_build_by_run_for_single_pos_arg',
 '_build_shapes_dict',
 '_call_has_mask_arg',
 '_call_has_training_arg',
 '_call_signature',
 '_called',
 '_check_load_own_variables',
 '_check_quantize_args',
 '_check_super_called',
 '_checkpoint_adapter',
 '_checkpoint_dependencies',
 '_clear_losses',
 '_convert_input_args',
 '_copy_trackable_to_cpu',
 '_default_s

In [357]:
modelnn.get_layer(index=0).units

64

In [358]:
modelnn.loss

'binary_crossentropy'

In [359]:
# df=pd.DataFrame({'a':[1,2,3],'b':[4,5,6]})

# columns=[('c','a'),('c','b')]

# df.columns=pd.MultiIndex.from_tuples(columns)

# df=pd.DataFrame({'a':[1,2,3],'b':[4,5,6]})

# columns=[('d','a'),('d','b')]
# df.columns=pd.MultiIndex.from_tuples(columns)

# df

In [360]:

Models = []
modelconfig = modelnn.get_config()
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 1', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layers': i, 
                   'Neurons': neurons, 
                   'Activation': activation})

ModelsDf = pd.DataFrame(Models)
# ModelsDf = ModelsDf.set_index('Model', append=True).swaplevel(0,1)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 652us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 677us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step


In [361]:
modelsoftmax = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [362]:
modelsoftmax.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelsoftmax.fit(x_train, keras.utils.to_categorical(y_train), epochs=10)


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 908us/step - accuracy: 0.8108 - loss: 0.4377
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 908us/step - accuracy: 0.8511 - loss: 0.3239
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 911us/step - accuracy: 0.8543 - loss: 0.2985
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 853us/step - accuracy: 0.8616 - loss: 0.2870
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - accuracy: 0.8693 - loss: 0.2764
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 896us/step - accuracy: 0.8672 - loss: 0.2715
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - accuracy: 0.8725 - loss: 0.2584
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 941us/step - accuracy: 0.8694 - loss: 0.2676
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step - accuracy: 0.8705 - loss: 0.2679
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step - accuracy: 0.8836 - loss: 0.2494


In [363]:
Models = []
modelconfig = modelnn.get_config()
modelnn = modelsoftmax
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 2'
                   , 'Optimizer': modelnn.optimizer.get_config()['name']
                   , 'Loss': modelnn.loss
                    , 'Accuracy': accuracy_score(y_test, np.argmax(modelsoftmax.predict(MS(x_test, intercept = False).fit_transform(x_test)), axis=1))
                   , 'Layers': i, 'Neurons': neurons
                   , 'Activation': activation})
    # Models.append({'Model': 'Neural Network 2', 'Optimizer': modelnn.optimizer.get_config()['name'], 'Loss': modelnn.loss, 'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 'Layers': i, 'Neurons': neurons, 'Activation': activation})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layers', 'Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 809us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step


In [364]:
modelsoftmax.evaluate(MS(x_test, intercept = False).fit_transform(x_test), keras.utils.to_categorical(y_test))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8548 - loss: 0.2836  


[0.28365808725357056, 0.8550000190734863]

In [365]:
print(accuracy_score(y_test, np.argmax(modelsoftmax.predict(MS(x_test, intercept = False).fit_transform(x_test)), axis=1)))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 698us/step
0.855


In [366]:
modelnn = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(1, activation='sigmoid')
])
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=100, verbose = 0)
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 930us/step - accuracy: 0.8568 - loss: 0.2727


[0.2757652997970581, 0.8565000295639038]

In [367]:
Models = []
modelconfig = modelnn.get_config()
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 3', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layers': i, 
                   'Neurons': neurons, 
                   'Activation': activation})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layers', 'Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step


In [368]:
modelnn = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dropout(0.5),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [369]:
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=100, verbose = 0)
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8524 - loss: 0.2908 


[0.2864534258842468, 0.8510000109672546]

In [370]:
Models = []
modelconfig = modelnn.get_config()
modelconfig['layers'][2]
# ModelsDf
for i in range(1, len(modelconfig['layers'])):
    layer_type = modelconfig['layers'][i]['class_name']
    if layer_type == 'Dropout':
        rate = modelconfig['layers'][i]['config']['rate']
        Models.append({'Model': 'Neural Network 4', 
                       'Optimizer': modelnn.optimizer.get_config()['name'], 
                       'Loss': modelnn.loss, 
                       'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                       'Layers': i, 
                       'Rate': rate})
        continue
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    Models.append({'Model': 'Neural Network 4', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layers': i, 
                   'Neurons': neurons, 
                   'Activation': activation})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layers', 'Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 645us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step


In [371]:
modelnn = keras.models.load_model('modelnn.keras')
accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


0.88

Note That the full data file is not included in the repository, as it is too large and that the code below takes over 30 mins to run. The code above just loads the model and calculates the accuracy score

In [372]:
# from astropy.io import ascii
# FullData = ascii.read(PathToRepo + '\\Data\\m12i_res7100_mhdcv.disk.ecsv', guess = False)
# FullData = FullData.to_pandas()

In [373]:
# FullData['IsMigrator'] = abs(FullData['Rcyl'] - FullData['Rcyl_form']) > 1.5
# FullData['IsMigratorInt'] = FullData['IsMigrator'].astype(int)
# FullData = FullData.drop(['rsph', 'x', 'y',  'vx', 'vy', 
#        'rsph_form', 'x_form', 'y_form', 'z_form', 'vx_form', 'vy_form',
#        'vz_form', 'Rcyl_form', 'phi_form', 'vRcyl_form', 'vphi_form', "IsMigrator"], axis = 1)

In [374]:
# Df = FullData
# x = Df.drop('IsMigratorInt', axis=1) 
# for i in range(0, x.shape[1]): #Standardizing the data
#     x.iloc[:,i] = (x.iloc[:, i] - x.iloc[:, i].mean())/x.iloc[:, i].std()

# x_train, x_test, y_train, y_test = train_test_split(x, Df['IsMigratorInt'], test_size=0.2, random_state=42)

In [375]:
# modelnn = keras.Sequential([
#     layers.Dense(16, activation='relu', input_shape=[x_train.shape[1]]),
#     layers.Dense(1, activation='sigmoid')
# ])
# modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# modelnn.fit(x_train, y_train, epochs=10)

In [376]:
Models = []

modelconfig = modelnn.get_config()
modelnn = keras.models.load_model('modelnn.keras')
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 5', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layers': i, 
                   'Neurons': neurons, 
                   'Activation': activation})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layers', 'Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step


C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [377]:
asd = ModelsDf

In [459]:
ModelsDf = asd

ModelsDf
ModelsDf.index = ModelsDf['Model']
ModelsDf = ModelsDf.drop('Model', axis = 1)

In [460]:
ModelsDf.index = ModelsDf.index.rename('Model')

In [461]:
ModelsDf.index = pd.MultiIndex.from_tuples([(ModelsDf.index[i],i) for i in range(0, ModelsDf.shape[0])])

In [462]:
ModelsDf['Loss'].replace("_", "\textunderscore ", regex=True, inplace=True)

C:\Users\lorentsen\AppData\Local\Temp\ipykernel_51116\2923490059.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ModelsDf['Loss'].replace("_", "\textunderscore ", regex=True, inplace=True)


In [463]:
ModelsDf.to_latex(PathToRepo + '\\Tables\\Models.tex', index_names = True, na_rep='-', multirow = True, float_format="%.2f", caption="Differnet neural networks", label="tab:RawData", position="h!")
ModelsDf

Optimizer                                     Loss  \
Neural Network 1 0       adam       binary\textunderscore crossentropy   
                 1       adam       binary\textunderscore crossentropy   
                 2       adam       binary\textunderscore crossentropy   
Neural Network 2 3       adam  categorical\textunderscore crossentropy   
                 4       adam  categorical\textunderscore crossentropy   
                 5       adam  categorical\textunderscore crossentropy   
Neural Network 3 6       adam       binary\textunderscore crossentropy   
                 7       adam       binary\textunderscore crossentropy   
Neural Network 4 8       adam       binary\textunderscore crossentropy   
                 9       adam       binary\textunderscore crossentropy   
                 10      adam       binary\textunderscore crossentropy   
                 11      adam       binary\textunderscore crossentropy   
                 12      adam       binary\textunderscore crossentropy   
Neural Network 5 13   rmsprop       binary\textunderscore crossentropy   
                 14   rmsprop       binary\textunderscore crossentropy   

                     Accuracy  Layers  Neurons Activation  Rate  
Neural Network 1 0     0.8695       1     64.0       relu   NaN  
                 1     0.8695       2     64.0       relu   NaN  
                 2     0.8695       3      1.0    sigmoid   NaN  
Neural Network 2 3     0.8550       1     64.0       relu   NaN  
                 4     0.8550       2     64.0       relu   NaN  
                 5     0.8550       3      1.0    sigmoid   NaN  
Neural Network 3 6     0.8565       1     16.0       relu   NaN  
                 7     0.8565       2      1.0    sigmoid   NaN  
Neural Network 4 8     0.8510       1     16.0       relu   NaN  
                 9     0.8510       2      NaN        NaN   0.5  
                 10    0.8510       3     16.0       relu   NaN  
                 11    0.8510       4      NaN        NaN   0.5  
                 12    0.8510       5      1.0    sigmoid   NaN  
Neural Network 5 13    0.8800       1     16.0       relu   NaN  
                 14    0.8800       2      1.0    sigmoid   NaN

In [382]:
# modelnn.save('modelnn.keras')